In [1]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
import Augmentor
import cv2
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import image
from tensorflow.keras.utils import array_to_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.client import device_lib; print(device_lib.list_local_devices())
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14453562481438365281
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7798259712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12152733939843995739
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
def AugOneImage(img, label, size=10):
    rotation_datagen = ImageDataGenerator(rotation_range = 10, 
                                      shear_range = 10,
                                      width_shift_range=0.01,
                                     height_shift_range=0.01,
                                     zoom_range=0.1)
#   imgの形状 =（バッチ数×縦×横×チャンネル数）
    img_iter=rotation_datagen.flow(img, batch_size = 1, seed = 0)
    aug_imgs=[]
    aug_labels=[]
    aug_imgs.append(img[0])
    aug_labels.append(label)
    size-=1
    for i, img_gen in enumerate(img_iter):
        if i == size:
          break
        aug_imgs.append(img_gen[0])
        aug_labels.append(label)
    return np.array(aug_imgs), np.array(aug_labels)

def AugAllImage(train_img, label, aug_size):
    aug_all_imgs=[]
    aug_all_labels=[]
    for img_data in zip(train_img, label):
        img=img_data[0].transpose(1, 2, 0)[np.newaxis, :, :, :]

        aug_imgs, aug_labels=AugOneImage(img, img_data[1], aug_size)
        aug_all_imgs.extend(aug_imgs)
        aug_all_labels.extend(aug_labels)
    return np.array(aug_all_imgs), np.array(aug_all_labels)

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [4]:
print(x_train.shape) 
print(y_train.shape)
print(x_test.shape) 
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
# 教師ラベルをonehotエンコード
enc = OneHotEncoder(categories="auto", sparse_output=False, dtype=np.float32)
y_train=enc.fit_transform(y_train[:, np.newaxis])
y_test=enc.fit_transform(y_test[:, np.newaxis])

In [6]:
X_train= x_train.reshape(-1, 1, 28, 28)
X_test= x_test.reshape(-1, 1, 28, 28)
print("train_data.shape=", X_train.shape)
print("train_label.shape=", y_train.shape)
print("test_data.shape=", X_test.shape)
print("test_label.shape=", y_test.shape)

train_data.shape= (60000, 1, 28, 28)
train_label.shape= (60000, 10)
test_data.shape= (10000, 1, 28, 28)
test_label.shape= (10000, 10)


## 訓練データ拡張
### ImageDataGenerator

In [25]:
aug_size=10
aug_imgs, aug_labels=AugAllImage(X_train, y_train, aug_size)

X_train=aug_imgs.transpose(0, 3, 1, 2)
y_train=aug_labels
print(X_train.shape, y_train.shape)

(600000, 1, 28, 28) (600000, 10)


### Augmentor

In [26]:
p = Augmentor.DataPipeline(X_train, y_train.argmax(axis=1).tolist())
p.random_distortion(probability=1, grid_width=1, grid_height=1, magnitude=7)
x_aug, y_aug=p.sample(X_train.shape[0])
x_aug=np.array(x_aug)
y_aug=np.array(y_aug).reshape(-1, 1)
oh_enc=OneHotEncoder(categories=[np.arange(y_train.shape[1]).tolist()], sparse=False)
y_aug=oh_enc.fit_transform(y_aug)
print(x_aug.shape, y_aug.shape)

(600000, 1, 28, 28) (600000, 10)


C:\Users\Arakawa\anaconda3\envs\jdla\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### ImageDataGeneratorとAugmentorを結合

In [27]:
X_train=np.vstack([X_train, x_aug])
y_train=np.vstack([y_train, y_aug])
print(X_train.shape, y_train.shape)

(1200000, 1, 28, 28) (1200000, 10)


## 前処理
### 正規化

In [7]:
X_train = (X_train - X_train.min()) / (X_train.max()- X_train.min())
X_train = X_train.astype('float32')

X_test = (X_test - X_test.min()) / (X_test.max()- X_test.min())
X_test = X_test.astype('float32')

## HDF5出力

In [8]:
f=h5py.File('data/auged_input_data.hdf5', 'w')
f.create_dataset('X_train', data=X_train)
f.create_dataset('X_test', data=X_test)
f.create_dataset('y_train', data=y_train)
f.create_dataset('y_test', data=y_test)
f.close()